In [1]:
import json
from pyspark import  SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import Word2Vec
from pyspark import Row
sqlContext = SQLContext(sc)  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38176)
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/py4j-0.9.2-py2.7.egg/py4j/java_gateway.py", line 713, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib64/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38176)
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/site-packages/py4j-0.9.2-py2.7.egg/py4j/java_gateway.py", line 713, in start
    self.socket.connect((self.address, self.port))
  File "/usr/lib64/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38176)
Traceback (most rec

NameError: name 'sc' is not defined

In [2]:
# handling issue with dot in bucket name
import ssl
if hasattr(ssl, '_create_unverified_context'):
   ssl._create_default_https_context = ssl._create_unverified_context


In [3]:
# aws credential
aws_access_key_id = ''
aws_secret_access_key = ''

In [4]:
import boto

In [5]:
conn = boto.connect_s3()

In [6]:
b = conn.get_bucket('han.2016.march')

In [7]:
k_list = [k.name for k in b.get_all_keys() ]
url_list = [ sc.textFile('s3n://{}:{}@han.2016.march/{}'.format(aws_access_key_id, aws_secret_access_key, k)) for k in k_list]
all_tweets = sc.union(url_list)

In [9]:
url_list[999]

MapPartitionsRDD[1999] at textFile at null:-1

In [10]:
output = []
for url in url_list:
    tweets = sqlContext.read.json(url)
    tweets.registerTempTable("tweets")
    sqlFilterCommand = "SELECT text FROM tweets WHERE (lang = 'en') AND "+"( text like '%trump%')"
    tweetsDF = sqlContext.sql(sqlFilterCommand).collect()
    if len(tweetsDF) != 0:
        output.extend(tweetsDF)

In [11]:
out = sc.parallelize(output)

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [13]:
def parseAndRemoveStopWords(text):
    t = text.replace(";"," ").replace(":"," ").replace('"',' ').replace('-',' ')
    t = t.replace(',',' ').replace('.',' ')
    t = t.lower().split(" ")
#     stop = stopwords.words('english')
    return [i for i in t if i not in stop]


In [14]:
prep_w2v = out.map(lambda x: x['text']).map(parseAndRemoveStopWords)

In [15]:
from pyspark.mllib.feature import Word2Vec


In [16]:
w2v = Word2Vec()

In [17]:
w2v_output = w2v.fit(prep_w2v)

In [18]:
w2v_output.findSynonyms('trump',10000)

[(u'kid', 0.39498921793081315),
 (u'would', 0.39485935274055395),
 (u'#kkktrump', 0.39374840796352706),
 (u'live', 0.38841990716940283),
 (u'america', 0.37060331732387353),
 (u'first', 0.36086675685374231),
 (u'lose', 0.3597226167355641),
 (u'today!', 0.35767082008065043),
 (u'24hrs', 0.34977145865615294),
 (u'campaign', 0.3451225897150566),
 (u'one', 0.34494202084515058),
 (u'#nevertrump\n#mapoli', 0.34286857181797986),
 (u'\U0001f611#danielforpres\u2026', 0.33688441168695005),
 (u'let', 0.32737243527558352),
 (u'believe', 0.32572269442158436),
 (u'actual', 0.32472242173884086),
 (u'serious', 0.31790397311320262),
 (u'ted', 0.31779130354248908),
 (u'marco', 0.31492913000103318),
 (u'@veggiejuless', 0.31232073223100904),
 (u'face', 0.31206119929710174),
 (u'kkk', 0.31168191959908814),
 (u'joke', 0.31043819077986229),
 (u'#trump', 0.30798145932823967),
 (u'baby', 0.30714835156818027),
 (u'decal', 0.30492532211734719),
 (u'president', 0.30264584452127935),
 (u'lets', 0.29872919231655126)

In [19]:
len(output)

1301

In [20]:
output[10
       ]

Row(text=u'@jaimedea20 @veggiejuless thats why you and your whole mexican food eating family need to vote for anyone but trump')